# We will try to make a Q Table for solving. 
We are going to disable slippery for reducing complications.

In [24]:
import gym
import numpy as np
import random
from time import sleep
from IPython.display import clear_output
from gym.envs.registration import register        # <<<--- To manipulate the registry of the FrozenLake Source to remove slipping 

# Lets make the basic agent from the last exercise

In [25]:
class Agent():
    def __init__(self, env):
        self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete
        #     ^^^^ <<< Takes 1 if discrete else 0
        
        if self.is_discrete:
            self.action_size = env.action_space.n
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_size = env.action_space.shape
        
    def get_action(self):
        if self.is_discrete:
            a = random.choice(range(self.action_size))
        else:             # vvvv <<< takes low high and shape of output as args
            a = np.random.uniform(self.action_low,
                                  self.action_high,
                                  self.action_size)
        return a

try:    
    register(
        id='FrozenLake-NoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery' : False},     # <<<--- Passing to register as a **kwarg... 
        max_episode_steps=100,                                  # Refer https://github.com/openai/gym/blob/master/gym/envs/__init__.py
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env = gym.make('FrozenLake-NoSlip-v0')
agent = Agent(env)
state = env.reset()

for _ in range(250):
    action = agent.get_action()
    state, reward, done, info = env.step(action)
    if done:
        break
    env.render()
    sleep(0.5)
    clear_output(wait=True)
env.close()

# We will now make the Q Agent

In [ ]:
class QAgent(Agent):
    def __init__(self, env):
        super(QAgent, self).__init__(env)
        self.state_size = env.observation_space.n
        self.make_table()
    def make_table(self):
        QTable = np.random.random([self.state_size, self.action_size])
    def get_action(self, state):
        q_state = self.QTable[state]
        action = np.argmax(q_state)
        return action
    def train(self, exp):
        state, action, next_state, done, reward = exp
            